<a href="https://colab.research.google.com/github/lijin-durairaj-code-mode/machine-learning/blob/main/movie_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import json

#sklearn
from sklearn.feature_extraction.text import CountVectorizer

#config
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
class movie_recommender:

  def __init__(self):
    pass

  def pull_data(self):
    credits=pd.read_csv('./sample_data/tmdb_5000_credits.csv')
    movies=pd.read_csv('./sample_data/tmdb_5000_movies.csv')
    _data=movies.merge(credits,left_on='id',right_on='movie_id')
    return _data

  def shred_arr(self,_arr,_name):
    _arr=pd.Series(json.loads(_arr))
    return ', '.join(_arr.apply(lambda x:x[_name]))

  def shred_crew(self,_d):
    _crew=[]
    for _a in json.loads(_d):
      _important_role=['Directing','Sound','Camera']
      if _a['department'] in _important_role:
        _crew.append(_a['name'])
    return ', '.join(_crew)

  def combine_shredded_words(self,_data):
    _columns=['genres','movie_id', 'keywords', 'original_language','original_title', 'overview', 'production_companies','cast', 'crew']
    _data['genres']=_data.loc[:,_columns]['genres'].apply(lambda x:self.shred_arr(x,'name'))
    _data['keywords']=_data.loc[:,_columns]['keywords'].apply(lambda x:self.shred_arr(x,'name'))
    _data['production_companies']=_data.loc[:,_columns]['production_companies'].apply(lambda x:self.shred_arr(x,'name'))
    # @@@_data['spoken_languages']=_data.loc[:,_columns]['spoken_languages'].apply(lambda x:shred_arr(x,'name'))
    _data['cast']=_data.loc[:,_columns]['cast'].apply(lambda x:self.shred_arr(x,'character'))
    _data['crew']=_data.loc[:,_columns]['crew'].apply(lambda x:self.shred_crew(x))
    return _data

  def format_data(self,_data):
    _columns=['genres','movie_id', 'keywords', 'original_language','original_title', 'overview', 'production_companies','cast', 'crew']
    _data['formated_data']=_data.loc[:,_columns].apply(lambda x:str(x['keywords'])+str(x['original_language'])+str(x['overview'])+str(x['production_companies'])+str(x['cast'])+str(x['crew']),axis=1)
    return _data.loc[:,['movie_id','original_title','formated_data']]

  def stemming(self,_data):
    from nltk.stem.porter import PorterStemmer
    ps=PorterStemmer()
    _data['formated_data']=_data['formated_data'].apply(lambda x:ps.stem(x))
    return _data

  def remove_stopwords(self,_data):
    import string
    import nltk
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    nltk.download('punkt')
    nltk.download('stopwords')

    _tokenized_d=word_tokenize(_data)
    _spcl_characters=word_tokenize(string.punctuation)
    #_removed_stp_words=[_w for _w in _tokenized_d if _w not in stopwords.words('english')]
    _rmvd_spl_chatracter=[w for w in _tokenized_d if w not in _spcl_characters]
    return ' '.join(_rmvd_spl_chatracter)

  def bag_of_words(self,_d):
    cv=CountVectorizer(max_features=5000,stop_words='english')
    _d['formated_data']=cv.fit_transform(_d['formated_data']).toarray()
    return _d

  def main(self):
    _d=self.pull_data()
    _shredded_d=self.combine_shredded_words(_d)
    format_d=self.format_data(_shredded_d)
    stemmed_d=self.stemming(format_d)
    stemmed_d['formated_data']=stemmed_d['formated_data'].apply(lambda x:self.remove_stopwords(x))
    return stemmed_d

mr=movie_recommender()
_d=mr.main()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  del sys.path[0]


Streaming output truncated to the last 5000 lines.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading packa

In [ ]:
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(_d['formated_data']).toarray()
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
_similarity=cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(_similarity[5])),reverse=True,key=lambda x:x[1])[0:6]

[(10, 0.9999999999999998),
 (14, 0.42461074172610996),
 (870, 0.39508721327563373),
 (1296, 0.38228206171752716),
 (2433, 0.3325918276996865),
 (813, 0.25300920190116793)]

In [ ]:
f1=_d.index==798
f2=_d.index==798
f3=_d.index==798
f4=_d.index==798
_d[f1 | f2 | f3 | ]['original_title']

798    The Internship
Name: original_title, dtype: object